In [101]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark import SparkContext


StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 106, Finished, Available)

In [5]:
%pip install wget

StatementMeta(, , -1, Finished, Available)

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=d1b57f8fd9bcc6906254982eda2610a3a3965362efc19c020c80c9c93b2f7c34
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [10]:
import wget
c = wget.download("https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/bing_covid-19_data/latest/bing_covid-19_data.parquet")

StatementMeta(, 6544388b-8012-476f-bafd-68f5d1284b29, 16, Finished, Available)

In [7]:
display(c)

StatementMeta(, 6544388b-8012-476f-bafd-68f5d1284b29, 13, Finished, Available)

'bing_covid-19_data.parquet'

In [65]:
covid = spark.read.parquet("Files/COVID/COVID.parquet").dropDuplicates(["id"])
# df now is a Spark DataFrame containing parquet data from "Files/COVID/COVID.parquet".
display(covid)

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 67, Finished, Available)

SynapseWidget(Synapse.DataFrame, b7faafaf-328a-4862-882c-67fd94fe4333)

In [67]:
covid.count()

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 69, Finished, Available)

4762923

In [68]:
covid.write.mode("overwrite").format("delta").saveAsTable("t_covid")

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 70, Finished, Available)

In [69]:
%%sql
select count(*) from t_covid

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 71, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [70]:
covid.printSchema()

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 72, Finished, Available)

root
 |-- id: integer (nullable = true)
 |-- updated: date (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- confirmed_change: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- deaths_change: short (nullable = true)
 |-- recovered: integer (nullable = true)
 |-- recovered_change: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- iso2: string (nullable = true)
 |-- iso3: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- admin_region_1: string (nullable = true)
 |-- iso_subdivision: string (nullable = true)
 |-- admin_region_2: string (nullable = true)
 |-- load_time: timestamp (nullable = true)



In [71]:
%%sql
drop table T_covid_sql

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 73, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [72]:
%%sql
CREATE TABLE IF NOT EXISTS T_COVID_SQL 
    (id BIGINT, 
     updated TIMESTAMP, 
     confirmed INT, 
     confirmed_change STRING, 
     deaths INT, 
     deaths_change INT, 
     recovered INT, 
     recovered_change INT, 
     latitude DOUBLE, 
     longitude DOUBLE, 
     iso2 STRING, 
     iso3 STRING,
     country_region STRING,
     admin_region_1 STRING,
     iso_subdivision STRING,
     admin_region_2 STRING,
     load_time TIMESTAMP)

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 74, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [73]:
%%sql

insert into t_covid_sql SELECT *  FROM t_covid

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 75, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [74]:
%%sql
select id, count(*) from t_covid_sql
group by id
having count(*) > 1

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 76, Finished, Available)

<Spark SQL result set with 0 rows and 2 fields>

In [75]:
#covid.write.mode("append").format("delta").saveAsTable("t_covid")
covid.write.mode("overwrite").format("delta").saveAsTable("t_covid")

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 77, Finished, Available)

In [76]:
%%sql 
select count(*) from t_covid

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 78, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [77]:
%%sql 
DELETE from t_covid where iso2 <> 'ES'

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 79, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [83]:
%%sql
select * from t_covid

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 85, Finished, Available)

<Spark SQL result set with 1000 rows and 17 fields>

In [78]:
%%sql
update t_covid set deaths = 0

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 80, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [79]:
%%sql

MERGE INTO t_covid_sql as d USING t_covid as m
  ON d.id = m.id
  WHEN MATCHED THEN 
    UPDATE SET *
  WHEN NOT MATCHED 
    THEN INSERT * ;

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 81, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [81]:
%%sql
DESCRIBE HISTORY t_covid_sql

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 83, Finished, Available)

<Spark SQL result set with 3 rows and 15 fields>

In [85]:
%%sql
SELECT * FROM t_covid_sql  WHERE ID IN (6889394,204567);

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 87, Finished, Available)

<Spark SQL result set with 2 rows and 17 fields>

In [93]:
%%sql
SELECT * FROM t_covid_sql VERSION AS OF 1 WHERE ID IN (6889394,204567);

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 96, Finished, Available)

<Spark SQL result set with 0 rows and 17 fields>

In [87]:
%%sql
RESTORE TABLE t_covid_sql TO VERSION AS OF 1;

SELECT * FROM t_covid_sql  WHERE ID IN (6889394,204567);

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 1 rows and 6 fields>

<Spark SQL result set with 2 rows and 17 fields>

In [95]:
%%sql
OPTIMIZE t_covid_sql;
-- as we vacuum with 0 hours, we need to remove the safety check:
set spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM t_covid_sql retain 0 hours;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

<Spark SQL result set with 1 rows and 2 fields>

<Spark SQL result set with 1 rows and 1 fields>

In [96]:
%%sql
DESCRIBE HISTORY t_covid_sql

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 101, Finished, Available)

<Spark SQL result set with 8 rows and 15 fields>

In [98]:
%%sql
SELECT * FROM t_covid_sql VERSION AS OF 2 WHERE ID IN (6889394,204567);

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 103, Finished, Available)

Error: Job aborted due to stage failure: Task 0 in stage 839.0 failed 4 times, most recent failure: Lost task 0.3 in stage 839.0 (TID 11136) (vm-9d187253 executor 3): org.apache.spark.SparkFileNotFoundException: Operation failed: "Not Found", 404, HEAD, http://onelake.dfs.fabric.microsoft.com/5205e934-5f3c-47a0-83c4-f187794a58cb/c8bcd2c1-a2f9-47ca-802f-8c3965a78815/Tables/t_covid_sql/part-00000-5f6f6f9e-d78f-4676-b2d8-cc614f6cdf26-c000.snappy.parquet?upn=false&action=getStatus&timeout=90
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:794)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:269)
	at org.apache.spar

In [102]:
covid2 = covid.withColumn("SecondID", col("id"))

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 107, Finished, Available)

In [104]:
covid2.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable("t_covid")

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 109, Finished, Available)

In [105]:
%%sql
SELECT * FROM t_covid WHERE ID IN (6889394,204567);

StatementMeta(, a41676c5-7c64-4152-8ffe-cd52760afdf3, 110, Finished, Available)

<Spark SQL result set with 3 rows and 18 fields>